In [4]:
import json
import itertools
import os
import random
from pathlib import Path
from time import sleep

import numpy as np
import pandas as pd
from dotenv import load_dotenv

from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials

diff_dataset = False

data_path = Path("../../data").resolve()
os.makedirs(data_path, exist_ok=True)

load_dotenv()
spotify = Spotify(client_credentials_manager=SpotifyClientCredentials(),
                  requests_timeout=10, retries=3)

In [5]:
def fetch_spotify_chart(filepath, num_samples=1000, seed=42):
    # currently: batshit crazy 4.3 GB spotify top200 chart CSV from kaggle: https://www.kaggle.com/general/232036
    # @TODO: series of fetching for random date from 1/1/2017 to present day.
    # url = "https://spotifycharts.com/regional/global/weekly/2018-10-12--2018-10-19/download"
    random.seed(seed)
    n = sum(1 for line in open(filepath)) - 1  # number of records in file (excludes header)
    s = num_samples  # desired sample size
    skip = sorted(random.sample(range(1, n + 1), n - s))  # the 0-indexed header will not be included in the skip list
    df = pd.read_csv(filepath, skiprows=skip)
    df["id"] = df["url"].apply(lambda url: url.split("/")[-1])
    return df


spotify_chart_df = fetch_spotify_chart("/Users/pez/Downloads/charts.csv", num_samples=15000, seed=42)

In [6]:
def extract_track_ids(df, id_key="id"):
    return list(df[(~df[id_key].isnull())]
                [id_key].to_dict().values())

# track ids from spotify top 200 daily charts selected at random
spotify_chart_track_ids = extract_track_ids(spotify_chart_df, "id")

# track ids from grammy nominations dataset
recording_nominations_df = pd.read_parquet(data_path / "raw/recording_nominations.pq")
recording_nominations_track_ids = extract_track_ids(recording_nominations_df, "spotify_track_id")

# track ids from chart2000 dataset
song_chart_df = pd.read_parquet(data_path / "raw/song_chart_by_month.pq")
song_chart_track_ids = extract_track_ids(song_chart_df, "spotify_track_id")

# track ids from random table
random_tracks_df = pd.read_csv(data_path / "random_track_ids.csv")
random_track_ids = extract_track_ids(random_tracks_df, "track_id")

# combine track ids and deduplicate
track_ids = list(set(
    recording_nominations_track_ids
    + song_chart_track_ids
    + spotify_chart_track_ids
    + random_track_ids
))

print(f"unique tracks: {len(track_ids)}")
print(f"spotify chart tracks: {len(set(spotify_chart_track_ids))}")
print(f"grammy tracks: {len(set(recording_nominations_track_ids))}")
print(f"chart2000 tracks: {len(set(song_chart_track_ids))}")
print(f"random tracks: {len(set(random_track_ids))}")

# how many new unique tracks have we added with additional random tracks?
track_ids_ = list(set(
    recording_nominations_track_ids
    + song_chart_track_ids
    + spotify_chart_track_ids
))
len(set(track_ids_) & set(random_track_ids))

unique tracks: 16129
spotify chart tracks: 8063
grammy tracks: 535
chart2000 tracks: 3379
random tracks: 4872


86

In [7]:
def fetch_bulk_track_features(track_ids):
    fetched_tracks = []
    for i in range(0, len(track_ids), 50):
        chunk = track_ids[i:i + 50]
        if i % 500 == 0:
            print(f"fetching track metadata: {i} ({i / len(track_ids) * 100:.2f})")
        tracks = spotify.tracks(chunk)["tracks"]
        sleep(.50)
        track_audio_features = spotify.audio_features(chunk)
        sleep(.50)
        for track, audio_features in zip(tracks, track_audio_features):
            track_ = {
                "id": track["id"],
                "name": track["name"],
                "album": track["album"]["name"],
                "album_id": track["album"]["id"],
                "artists": json.dumps([a["name"] for a in track["artists"]]),
                "artist_ids": json.dumps([a["id"] for a in track["artists"]]),
                "track_number": track["track_number"],
                "disc_number": track["disc_number"],
                "explicit": track["explicit"],
                "duration_ms": track["duration_ms"],
                "year": int(track["album"]["release_date"][0:4]),
                "release_date": track["album"]["release_date"],
                "popularity": track["popularity"],
                "audio_features": 0,
                "isrc": track.get("external_ids", {}).get("isrc")
            }
            if audio_features is not None:
                track_.update({
                    "audio_features": 1,
                    "danceability": audio_features["danceability"],
                    "energy": audio_features["energy"],
                    "key": audio_features["key"],
                    "loudness": audio_features["loudness"],
                    "mode": audio_features["mode"],
                    "speechiness": audio_features["speechiness"],
                    "acousticness": audio_features["acousticness"],
                    "instrumentalness": audio_features["instrumentalness"],
                    "liveness": audio_features["liveness"],
                    "valence": audio_features["valence"],
                    "tempo": audio_features["tempo"],
                    "time_signature": audio_features["time_signature"],
                })
            fetched_tracks.append(track_)
    return pd.DataFrame(fetched_tracks)


tracks_df = fetch_bulk_track_features(track_ids)
print(tracks_df.shape)
tracks_df.head()

fetching track metadata: 0 (0.00)
fetching track metadata: 500 (3.10)
fetching track metadata: 1000 (6.20)
fetching track metadata: 1500 (9.30)
fetching track metadata: 2000 (12.40)
fetching track metadata: 2500 (15.50)
fetching track metadata: 3000 (18.60)
fetching track metadata: 3500 (21.70)
fetching track metadata: 4000 (24.80)
fetching track metadata: 4500 (27.90)
fetching track metadata: 5000 (31.00)
fetching track metadata: 5500 (34.10)
fetching track metadata: 6000 (37.20)
fetching track metadata: 6500 (40.30)
fetching track metadata: 7000 (43.40)
fetching track metadata: 7500 (46.50)
fetching track metadata: 8000 (49.60)
fetching track metadata: 8500 (52.70)
fetching track metadata: 9000 (55.80)
fetching track metadata: 9500 (58.90)
fetching track metadata: 10000 (62.00)
fetching track metadata: 10500 (65.10)
fetching track metadata: 11000 (68.20)
fetching track metadata: 11500 (71.30)
fetching track metadata: 12000 (74.40)
fetching track metadata: 12500 (77.50)
fetching track

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,duration_ms,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,10iesoGb4mCYTcur1QfdO9,JCVD,Rien 100 Rien,2GLuHlc49dJKY8yzxUZb8p,"[""Jul""]","[""3IW7ScrzXmPvZhB27hmfgy""]",2,1,True,194266,...,7.0,-5.873,0.0,0.0340,0.479,0.000003,0.068,0.895,130.050,4.0
1,0DlHQxrVijRqVUfuS6PKaY,Deixa de Onda (Porra Nenhuma),Deixa de Onda (Porra Nenhuma),1fjkJADMiiZWMDusZSMNix,"[""DENNIS"", ""Xam\u00e3"", ""LUDMILLA""]","[""6xlRSRMLgZbsSNd0BMobwy"", ""5YwzDz4RJfTiMHS4td...",1,1,True,184999,...,9.0,-4.622,1.0,0.0469,0.290,0.000000,0.312,0.654,135.036,4.0
2,2QtM6ZVLx13TBOO1iwmXYy,Cubra Libre Bar - Café Buddha del Mar Mix,Hotel Paradise del Mar (Chill Out Lounge Café ...,3wsWiiFbTF3TpC41yStCTL,"[""Mark Bareilles""]","[""22PKRoZlEMpKexRXSLStVP""]",12,1,False,332068,...,9.0,-13.140,0.0,0.0392,0.823,0.912000,0.111,0.450,79.988,4.0
3,27hFQQS3cVUmIK3ser5bpu,More Than Gravity,More Than Gravity,4kcSIHj40oMyfHgMG6XAmO,"[""Colin & Caroline""]","[""2Y4125mDrgQyPD7dt76Rtm""]",1,1,False,266078,...,4.0,-9.648,1.0,0.0268,0.715,0.000039,0.106,0.217,110.969,4.0
4,1YcqYmviKagJ68DOmjSWQW,Tojo Tojo,Tojo Tojo,58NV8EvCFPHWwi7HHdVqSy,"[""Robin Zoot"", ""Nik Tendo"", ""Karlo""]","[""28oxY7ZfDt1BJFf8FVfpEh"", ""6gvVFtkTV4OCyQSCzG...",1,1,True,197010,...,2.0,-6.595,1.0,0.4650,0.186,0.000000,0.109,0.732,147.586,4.0


In [8]:
def with_track_charting_summary(df, song_chart_df):
    song_chart_df = song_chart_df.rename(columns={
        "spotify_artist_id": "artist_id",
        "spotify_track_id": "id"
    })
    song_chart_df["chart_months"] = 1
    song_chart_df = (
        song_chart_df
            .sort_values(by=["id"])
            .groupby(["id"])
            .aggregate({
                "chart_months": "sum",
                "indicativerevenue": "sum",
                "position": ["min", "mean", "median"],
            })
            .reset_index()
            .set_axis(["id", "track_chart_months", "track_chart_indicativerevenue", "track_chart_peak", "track_chart_mean", "track_chart_median"], axis=1)
    )

    df = pd.merge(df, song_chart_df, on="id", how="left")
    df["track_chart_months"] = df["track_chart_months"].fillna(0)
    df["track_chart_indicativerevenue"] = df["track_chart_indicativerevenue"].fillna(0)
    return df


drop_columns = [col for col in tracks_df.columns if col.startswith("track_chart_")]
tracks_df = tracks_df.drop(columns=drop_columns)

song_chart_df = pd.read_parquet(data_path / "raw/song_chart_by_month.pq")
tracks_df = with_track_charting_summary(tracks_df, song_chart_df)
print(tracks_df.shape)
tracks_df.head()

(16129, 32)


,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,duration_ms,...,instrumentalness,liveness,valence,tempo,time_signature,track_chart_months,track_chart_indicativerevenue,track_chart_peak,track_chart_mean,track_chart_median
0,10iesoGb4mCYTcur1QfdO9,JCVD,Rien 100 Rien,2GLuHlc49dJKY8yzxUZb8p,"[""Jul""]","[""3IW7ScrzXmPvZhB27hmfgy""]",2,1,True,194266,...,0.000003,0.068,0.895,130.050,4.0,0.0,0.0,NaN,NaN,NaN
1,0DlHQxrVijRqVUfuS6PKaY,Deixa de Onda (Porra Nenhuma),Deixa de Onda (Porra Nenhuma),1fjkJADMiiZWMDusZSMNix,"[""DENNIS"", ""Xam\u00e3"", ""LUDMILLA""]","[""6xlRSRMLgZbsSNd0BMobwy"", ""5YwzDz4RJfTiMHS4td...",1,1,True,184999,...,0.000000,0.312,0.654,135.036,4.0,0.0,0.0,NaN,NaN,NaN
2,2QtM6ZVLx13TBOO1iwmXYy,Cubra Libre Bar - Café Buddha del Mar Mix,Hotel Paradise del Mar (Chill Out Lounge Café ...,3wsWiiFbTF3TpC41yStCTL,"[""Mark Bareilles""]","[""22PKRoZlEMpKexRXSLStVP""]",12,1,False,332068,...,0.912000,0.111,0.450,79.988,4.0,0.0,0.0,NaN,NaN,NaN
3,27hFQQS3cVUmIK3ser5bpu,More Than Gravity,More Than Gravity,4kcSIHj40oMyfHgMG6XAmO,"[""Colin & Caroline""]","[""2Y4125mDrgQyPD7dt76Rtm""]",1,1,False,266078,...,0.000039,0.106,0.217,110.969,4.0,0.0,0.0,NaN,NaN,NaN
4,1YcqYmviKagJ68DOmjSWQW,Tojo Tojo,Tojo Tojo,58NV8EvCFPHWwi7HHdVqSy,"[""Robin Zoot"", ""Nik Tendo"", ""Karlo""]","[""28oxY7ZfDt1BJFf8FVfpEh"", ""6gvVFtkTV4OCyQSCzG...",1,1,True,197010,...,0.000000,0.109,0.732,147.586,4.0,0.0,0.0,NaN,NaN,NaN


In [9]:
def with_recording_nominations(df, recording_nominations_df):
    recording_nominations_df = (
        recording_nominations_df[[
                "award_category",
                "award_nominee",
                "award_winner",
                "spotify_track_id",
            ]]
            .groupby(["spotify_track_id"])
            .aggregate({
                "award_category": "max",
                "award_nominee": "sum",
                "award_winner": "sum",
            })
            .reset_index()
            .drop_duplicates(subset=["spotify_track_id"])
            .rename(columns={
                "spotify_track_id": "id",
                "award_category": "recording_award_category",
                "award_nominee": "recording_award_nominee",
                "award_winner": "recording_award_winner",
            })
    )
    df = pd.merge(df, recording_nominations_df, on="id", how="outer")
    df["recording_award_nominee"] = df["recording_award_nominee"].fillna(0)
    df["recording_award_winner"] = df["recording_award_winner"].fillna(0)
    return df


drop_columns = [col for col in tracks_df.columns if col.startswith("recording_award_")]
tracks_df = tracks_df.drop(columns=drop_columns)

recording_nominations_df = pd.read_parquet(data_path / "raw/recording_nominations.pq")
tracks_df = with_recording_nominations(tracks_df, recording_nominations_df)
print(tracks_df.shape)
tracks_df.head()

(16129, 35)


,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,duration_ms,...,tempo,time_signature,track_chart_months,track_chart_indicativerevenue,track_chart_peak,track_chart_mean,track_chart_median,recording_award_category,recording_award_nominee,recording_award_winner
0,10iesoGb4mCYTcur1QfdO9,JCVD,Rien 100 Rien,2GLuHlc49dJKY8yzxUZb8p,"[""Jul""]","[""3IW7ScrzXmPvZhB27hmfgy""]",2,1,True,194266,...,130.050,4.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0
1,0DlHQxrVijRqVUfuS6PKaY,Deixa de Onda (Porra Nenhuma),Deixa de Onda (Porra Nenhuma),1fjkJADMiiZWMDusZSMNix,"[""DENNIS"", ""Xam\u00e3"", ""LUDMILLA""]","[""6xlRSRMLgZbsSNd0BMobwy"", ""5YwzDz4RJfTiMHS4td...",1,1,True,184999,...,135.036,4.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0
2,2QtM6ZVLx13TBOO1iwmXYy,Cubra Libre Bar - Café Buddha del Mar Mix,Hotel Paradise del Mar (Chill Out Lounge Café ...,3wsWiiFbTF3TpC41yStCTL,"[""Mark Bareilles""]","[""22PKRoZlEMpKexRXSLStVP""]",12,1,False,332068,...,79.988,4.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0
3,27hFQQS3cVUmIK3ser5bpu,More Than Gravity,More Than Gravity,4kcSIHj40oMyfHgMG6XAmO,"[""Colin & Caroline""]","[""2Y4125mDrgQyPD7dt76Rtm""]",1,1,False,266078,...,110.969,4.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0
4,1YcqYmviKagJ68DOmjSWQW,Tojo Tojo,Tojo Tojo,58NV8EvCFPHWwi7HHdVqSy,"[""Robin Zoot"", ""Nik Tendo"", ""Karlo""]","[""28oxY7ZfDt1BJFf8FVfpEh"", ""6gvVFtkTV4OCyQSCzG...",1,1,True,197010,...,147.586,4.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0


In [10]:
def with_album_nominations(df, album_nominations_df):
    album_nominations_df = (
        album_nominations_df[[
            "award_category",
            "award_nominee",
            "award_winner",
            "spotify_album_id",
        ]]
        .groupby(["spotify_album_id"])
        .aggregate({
            "award_category": "max",
            "award_nominee": "sum",
            "award_winner": "sum",
        })
        .reset_index()
        .drop_duplicates(subset=["spotify_album_id"])
        .rename(columns={
            "spotify_album_id": "album_id",
            "award_category": "album_award_category",
            "award_nominee": "album_award_nominee",
            "award_winner": "album_award_winner",
        })
    )
    return (
        pd.merge(df, album_nominations_df, on="album_id",  how="left")
    )


album_nominations_df = pd.read_parquet(data_path / "raw/album_nominations.pq")
tracks_df = with_album_nominations(tracks_df, album_nominations_df)
print(tracks_df.shape)
tracks_df.head()

(16129, 38)


,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,duration_ms,...,track_chart_indicativerevenue,track_chart_peak,track_chart_mean,track_chart_median,recording_award_category,recording_award_nominee,recording_award_winner,album_award_category,album_award_nominee,album_award_winner
0,10iesoGb4mCYTcur1QfdO9,JCVD,Rien 100 Rien,2GLuHlc49dJKY8yzxUZb8p,"[""Jul""]","[""3IW7ScrzXmPvZhB27hmfgy""]",2,1,True,194266,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
1,0DlHQxrVijRqVUfuS6PKaY,Deixa de Onda (Porra Nenhuma),Deixa de Onda (Porra Nenhuma),1fjkJADMiiZWMDusZSMNix,"[""DENNIS"", ""Xam\u00e3"", ""LUDMILLA""]","[""6xlRSRMLgZbsSNd0BMobwy"", ""5YwzDz4RJfTiMHS4td...",1,1,True,184999,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
2,2QtM6ZVLx13TBOO1iwmXYy,Cubra Libre Bar - Café Buddha del Mar Mix,Hotel Paradise del Mar (Chill Out Lounge Café ...,3wsWiiFbTF3TpC41yStCTL,"[""Mark Bareilles""]","[""22PKRoZlEMpKexRXSLStVP""]",12,1,False,332068,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
3,27hFQQS3cVUmIK3ser5bpu,More Than Gravity,More Than Gravity,4kcSIHj40oMyfHgMG6XAmO,"[""Colin & Caroline""]","[""2Y4125mDrgQyPD7dt76Rtm""]",1,1,False,266078,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
4,1YcqYmviKagJ68DOmjSWQW,Tojo Tojo,Tojo Tojo,58NV8EvCFPHWwi7HHdVqSy,"[""Robin Zoot"", ""Nik Tendo"", ""Karlo""]","[""28oxY7ZfDt1BJFf8FVfpEh"", ""6gvVFtkTV4OCyQSCzG...",1,1,True,197010,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN


In [11]:
tracks_df["num_artists"] = tracks_df["artist_ids"].apply(json.loads).apply(len)

In [12]:
def lookup(artist_df, artist_ids):
    artist_ids = json.loads(artist_ids.replace("'", "\""))
    artists_ = []
    for artist_id in artist_ids:
        matches = artist_df[artist_df["id"] == artist_id]
        if len(matches) > 0:
            artists_.append(matches.to_dict('records')[0])
    return artists_


def with_artist_summary(df, artist_df):
    df = df.copy()
    # df["num_artists"] = df.apply(lambda r: len(df["artist_ids"]), axis=1)
    df["artists"] = df.apply(lambda r: lookup(artist_df, r["artist_ids"]), axis=1)
    df["artist_matched"] = df.apply(lambda r: len(r["artists"]) > 0, axis=1)
    df["artist_names"] = df.apply(lambda r: [a["name"] for a in r["artists"]], axis=1)
    df["artist_names"] = df.apply(lambda r: ", ".join(r["artist_names"]), axis=1)
    df["artist_genres"] = df.apply(lambda r: [a["genres"] for a in r["artists"]], axis=1)
    df["artist_genres"] = df.apply(lambda r: list(itertools.chain(*r["artist_genres"])), axis=1)
    df["artist_genres"] = df.apply(lambda r: r["artist_genres"] if r["artist_genres"] else np.nan, axis=1)
    # df["artist_genres"] = df.apply(lambda r: "; ".join(list(itertools.chain(*r["artist_genres"]))), axis=1)
    df["artist_popularity"] = df.apply(lambda r: [a["popularity"] for a in r["artists"]], axis=1)
    df["artist_popularity"] = df.apply(lambda r: max(r["artist_popularity"]) if r["artist_popularity"] else 0, axis=1)
    df["artist_first_release"] = df.apply(lambda r: [a["first_release"] for a in r["artists"]], axis=1)
    df["artist_first_release"] = df.apply(lambda r: min(r["artist_first_release"]) if r["artist_first_release"] else np.nan, axis=1)
    # df["artist_last_release"] = df.apply(lambda r: [a["last_release"] for a in r["artists"]], axis=1)
    # df[f"artist_last_release"] = df.apply(lambda r: max(df["artist_last_release"])
    df["artist_lifetime_releases"] = df.apply(lambda r: [a["total_releases"] for a in r["artists"]], axis=1)
    df["artist_lifetime_releases"] = df.apply(lambda r: sum(m for m in r["artist_lifetime_releases"] if m is not None), axis=1).fillna(0)
    df["artist_gender"] = df.apply(lambda r: [a["gender"] for a in r["artists"]], axis=1)
    #df["artist_gender"] = df.apply(lambda r: r["artist_gender"][0] if r["artist_gender"] else np.nan, axis=1)
    df["artist_country"] = df.apply(lambda r:[a["country"] for a in r["artists"]], axis=1)
    df["artist_country"] = df.apply(lambda r: r["artist_country"][0] if r["artist_country"] else np.nan, axis=1)
    df["artist_hometown"] = df.apply(lambda r: [a["hometown"] for a in r["artists"]], axis=1)
    df["artist_hometown"] = df.apply(lambda r: r["artist_hometown"][0] if r["artist_hometown"] else np.nan, axis=1)
    df["artist_first_win"] = df.apply(lambda r: [a["first_win"] for a in r["artists"]], axis=1)
    df["artist_first_win"] = df.apply(lambda r: min(r["artist_first_win"]) if r["artist_first_win"] else np.nan, axis=1)
    df["artist_first_nomination"] = df.apply(lambda r: [a["first_nomination"] for a in r["artists"]], axis=1)
    df["artist_first_nomination"] = df.apply(lambda r: min(r["artist_first_nomination"]) if r["artist_first_nomination"] else np.nan, axis=1)
    # df["artist_last_win"] = df.apply(lambda r: [a["last_win"] for a in r["artists"]], axis=1)
    # df["artist_last_win"] = df.apply(lambda r: max(df["artist_last_win"])
    # df["artist_last_nomination"] = df.apply(lambda r: [a["last_nomination"] for a in r["artists"]], axis=1)
    # df["artist_last_nomination"] = df.apply(lambda r: max(df["artist_last_nomination"])
    df["artist_lifetime_wins"] = df.apply(lambda r: [a["total_wins"] for a in r["artists"]], axis=1)
    df["artist_lifetime_wins"] = df.apply(lambda r: sum(m for m in r["artist_lifetime_wins"] if m is not None), axis=1).fillna(0)
    df["artist_lifetime_nominations"] = df.apply(lambda r: [a["total_nominations"] for a in r["artists"]], axis=1)
    df["artist_lifetime_nominations"] = df.apply(lambda r: sum(m for m in r["artist_lifetime_nominations"] if m is not None), axis=1).fillna(0)
    df["artist_lifetime_chart_months"] = df.apply(lambda r: [a["chart_months"] for a in r["artists"]], axis=1)
    df["artist_lifetime_chart_months"] = df.apply(lambda r: sum(r["artist_lifetime_chart_months"]), axis=1).fillna(0)
    df["artist_lifetime_revenue"] = df.apply(lambda r: [a["total_revenue"] for a in r["artists"]], axis=1)
    df["artist_lifetime_revenue"] = df.apply(lambda r: sum(m for m in r["artist_lifetime_revenue"] if m is not None), axis=1).fillna(0)
    df["artist_first_chart_year"] = df.apply(lambda r: [a["first_chart_year"] for a in r["artists"]], axis=1)
    df["artist_first_chart_year"] = df.apply(lambda r: min(r["artist_first_chart_year"]) if r["artist_first_chart_year"] else np.nan, axis=1)
    # df["artist_last_chart_year"] = df.apply(lambda r: [a["last_chart_year"] for a in r["artists"]], axis=1)
    # df["artist_last_chart_year"] = df.apply(lambda r: max(df["artist_last_chart_year"])
    df["artist_chart_tracks"] = df.apply(lambda r: [a["chart_tracks"] for a in r["artists"]], axis=1)
    df["artist_chart_tracks"] = df.apply(lambda r: max(r["artist_chart_tracks"]) if r["artist_chart_tracks"] else 0, axis=1).fillna(0)
    df["artist_chart_peak"] = df.apply(lambda r: [a["chart_peak"] for a in r["artists"]], axis=1)
    df["artist_chart_peak"] = df.apply(lambda r: min(r["artist_chart_peak"]) if r["artist_chart_peak"] else 0, axis=1)
    df["artist_nominations"] = df.apply(lambda r: [a.get(f"grammy_nominated_{r['year'] - 1}") for a in r["artists"]], axis=1)
    df["artist_nominations"] = df.apply(lambda r: sum(m for m in r["artist_nominations"] if m is not None), axis=1).fillna(0)
    df["artist_wins"] = df.apply(lambda r: [a.get(f"grammy_won_{r['year'] - 1}") for a in r["artists"]], axis=1)
    df["artist_wins"] = df.apply(lambda r: sum(m for m in r["artist_wins"] if m is not None), axis=1).fillna(0)
    df["artist_releases"] = df.apply(lambda r: [a.get(f"releases_{r['year'] - 1}") for a in r["artists"]], axis=1)
    df["artist_releases"] = df.apply(lambda r: sum(m for m in r["artist_releases"] if m is not None), axis=1).fillna(0)
    df["artist_chart_months"] = df.apply(lambda r: [a.get(f"chart_months_{r['year'] - 1}") for a in r["artists"]], axis=1)
    df["artist_chart_months"] = df.apply(lambda r: sum(m for m in r["artist_chart_months"] if m is not None), axis=1).fillna(0)
    df["artist_chart_months_recently"] = df.apply(lambda r: [a.get(f"chart_months_recently_{r['year'] - 1}") for a in r["artists"]], axis=1)
    df["artist_chart_months_recently"] = df.apply(lambda r: sum(m for m in r["artist_chart_months_recently"] if m is not None), axis=1).fillna(0)
    df["artist_releases_recently"] = df.apply(lambda r:  [a.get(f"releases_{r['year'] - 1}", 0) - a.get(f"releases_{r['year'] - 4}") for a in r["artists"]], axis=1)
    df["artist_releases_recently"] = df.apply(lambda r: sum(m for m in r["artist_releases_recently"] if m is not None), axis=1).fillna(0)
    df["artists"] = df["artist_names"]
    return df


artist_df = pd.read_parquet(data_path / "artist_summary.pq")
tracks_df = with_artist_summary(tracks_df, artist_df)
print(tracks_df.shape)
print(tracks_df[tracks_df["artist_matched"] == 0].shape)
tracks_df.head()

(16129, 63)
(759, 63)


,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,duration_ms,...,artist_lifetime_revenue,artist_first_chart_year,artist_chart_tracks,artist_chart_peak,artist_nominations,artist_wins,artist_releases,artist_chart_months,artist_chart_months_recently,artist_releases_recently
0,10iesoGb4mCYTcur1QfdO9,JCVD,Rien 100 Rien,2GLuHlc49dJKY8yzxUZb8p,Jul,"[""3IW7ScrzXmPvZhB27hmfgy""]",2,1,True,194266,...,0.0,0.0,0.0,0.0,0.0,0.0,39.0,0.0,0.0,33.0
1,0DlHQxrVijRqVUfuS6PKaY,Deixa de Onda (Porra Nenhuma),Deixa de Onda (Porra Nenhuma),1fjkJADMiiZWMDusZSMNix,"DENNIS, Xamã, LUDMILLA","[""6xlRSRMLgZbsSNd0BMobwy"", ""5YwzDz4RJfTiMHS4td...",1,1,True,184999,...,0.0,0.0,0.0,0.0,0.0,0.0,158.0,0.0,0.0,118.0
2,2QtM6ZVLx13TBOO1iwmXYy,Cubra Libre Bar - Café Buddha del Mar Mix,Hotel Paradise del Mar (Chill Out Lounge Café ...,3wsWiiFbTF3TpC41yStCTL,Mark Bareilles,"[""22PKRoZlEMpKexRXSLStVP""]",12,1,False,332068,...,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,27hFQQS3cVUmIK3ser5bpu,More Than Gravity,More Than Gravity,4kcSIHj40oMyfHgMG6XAmO,Colin & Caroline,"[""2Y4125mDrgQyPD7dt76Rtm""]",1,1,False,266078,...,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,1YcqYmviKagJ68DOmjSWQW,Tojo Tojo,Tojo Tojo,58NV8EvCFPHWwi7HHdVqSy,"Nik Tendo, Karlo","[""28oxY7ZfDt1BJFf8FVfpEh"", ""6gvVFtkTV4OCyQSCzG...",1,1,True,197010,...,0.0,0.0,0.0,0.0,0.0,0.0,81.0,0.0,0.0,68.0


In [13]:
def resolve_genre(genres, genre_weights):
    genre = sorted(genres, key=genre_weights.get)[0]\
        if genres is not None and isinstance(genres, list) and len(genres) > 0\
        else "other"
    # print(f"{genres} => {genre}")
    return genre


def with_genres_resolved(tracks_df, artist_df):
    genre_weights = artist_df["genres"].explode().value_counts().to_dict()
    genre_weight_values = list(genre_weights.values())
    genre_weight_cutoff = np.average(genre_weight_values) + 1.5 * np.std(genre_weight_values)
    genre_weights = dict((k, v if v > genre_weight_cutoff else 0) for k, v in genre_weights.items())
    tracks_df["genre"] = tracks_df["artist_genres"].apply(lambda genres: resolve_genre(genres, genre_weights))
    return tracks_df


# tracks_df = tracks_df.drop(columns=["genre"])

artist_df = pd.read_parquet(data_path / "artist_summary.pq")
tracks_df = with_genres_resolved(tracks_df, artist_df)
print(tracks_df[tracks_df["artist_matched"] == 0].shape)
print(tracks_df.shape)
tracks_df.head()

(759, 64)
(16129, 64)


,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,duration_ms,...,artist_first_chart_year,artist_chart_tracks,artist_chart_peak,artist_nominations,artist_wins,artist_releases,artist_chart_months,artist_chart_months_recently,artist_releases_recently,genre
0,10iesoGb4mCYTcur1QfdO9,JCVD,Rien 100 Rien,2GLuHlc49dJKY8yzxUZb8p,Jul,"[""3IW7ScrzXmPvZhB27hmfgy""]",2,1,True,194266,...,0.0,0.0,0.0,0.0,0.0,39.0,0.0,0.0,33.0,rap marseille
1,0DlHQxrVijRqVUfuS6PKaY,Deixa de Onda (Porra Nenhuma),Deixa de Onda (Porra Nenhuma),1fjkJADMiiZWMDusZSMNix,"DENNIS, Xamã, LUDMILLA","[""6xlRSRMLgZbsSNd0BMobwy"", ""5YwzDz4RJfTiMHS4td...",1,1,True,184999,...,0.0,0.0,0.0,0.0,0.0,158.0,0.0,0.0,118.0,funk das antigas
2,2QtM6ZVLx13TBOO1iwmXYy,Cubra Libre Bar - Café Buddha del Mar Mix,Hotel Paradise del Mar (Chill Out Lounge Café ...,3wsWiiFbTF3TpC41yStCTL,Mark Bareilles,"[""22PKRoZlEMpKexRXSLStVP""]",12,1,False,332068,...,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,other
3,27hFQQS3cVUmIK3ser5bpu,More Than Gravity,More Than Gravity,4kcSIHj40oMyfHgMG6XAmO,Colin & Caroline,"[""2Y4125mDrgQyPD7dt76Rtm""]",1,1,False,266078,...,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,acoustic cover
4,1YcqYmviKagJ68DOmjSWQW,Tojo Tojo,Tojo Tojo,58NV8EvCFPHWwi7HHdVqSy,"Nik Tendo, Karlo","[""28oxY7ZfDt1BJFf8FVfpEh"", ""6gvVFtkTV4OCyQSCzG...",1,1,True,197010,...,0.0,0.0,0.0,0.0,0.0,81.0,0.0,0.0,68.0,czsk hip hop


In [14]:
def with_labels(df, labels_df):
    labels_df = labels_df.drop_duplicates(["album_id"])
    # labels_df = labels_df.drop(columns=["id", "artist_ids"])
    labels_df["label_matched"] = 1
    df = pd.merge(df, labels_df, on="album_id", how="left")
    return df
    df["label_matched"] = df["label_matched"].fillna(0)
    # df["major_label"] = df["major_label"].fillna(0)
    return df


labels_df = pd.read_parquet(data_path / "labels/albums_with_annotated_labels.pq")
tracks_df = with_labels(tracks_df, labels_df)
print(tracks_df.shape)
print(tracks_df[tracks_df["label_matched"] == 0].shape)
tracks_df.head()

(16129, 67)
(0, 67)


,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,duration_ms,...,artist_nominations,artist_wins,artist_releases,artist_chart_months,artist_chart_months_recently,artist_releases_recently,genre,label_name,major_label,label_matched
0,10iesoGb4mCYTcur1QfdO9,JCVD,Rien 100 Rien,2GLuHlc49dJKY8yzxUZb8p,Jul,"[""3IW7ScrzXmPvZhB27hmfgy""]",2,1,True,194266,...,0.0,0.0,39.0,0.0,0.0,33.0,rap marseille,D’Or et de Platine,0.0,1.0
1,0DlHQxrVijRqVUfuS6PKaY,Deixa de Onda (Porra Nenhuma),Deixa de Onda (Porra Nenhuma),1fjkJADMiiZWMDusZSMNix,"DENNIS, Xamã, LUDMILLA","[""6xlRSRMLgZbsSNd0BMobwy"", ""5YwzDz4RJfTiMHS4td...",1,1,True,184999,...,0.0,0.0,158.0,0.0,0.0,118.0,funk das antigas,Sony Music,1.0,1.0
2,2QtM6ZVLx13TBOO1iwmXYy,Cubra Libre Bar - Café Buddha del Mar Mix,Hotel Paradise del Mar (Chill Out Lounge Café ...,3wsWiiFbTF3TpC41yStCTL,Mark Bareilles,"[""22PKRoZlEMpKexRXSLStVP""]",12,1,False,332068,...,0.0,0.0,0.0,0.0,0.0,0.0,other,NaN,NaN,NaN
3,27hFQQS3cVUmIK3ser5bpu,More Than Gravity,More Than Gravity,4kcSIHj40oMyfHgMG6XAmO,Colin & Caroline,"[""2Y4125mDrgQyPD7dt76Rtm""]",1,1,False,266078,...,0.0,0.0,0.0,0.0,0.0,0.0,acoustic cover,NaN,NaN,NaN
4,1YcqYmviKagJ68DOmjSWQW,Tojo Tojo,Tojo Tojo,58NV8EvCFPHWwi7HHdVqSy,"Nik Tendo, Karlo","[""28oxY7ZfDt1BJFf8FVfpEh"", ""6gvVFtkTV4OCyQSCzG...",1,1,True,197010,...,0.0,0.0,81.0,0.0,0.0,68.0,czsk hip hop,W.C. Records,0.0,1.0


In [15]:
tracks_df = tracks_df.rename(columns={"chart_months": "track_chart_months"})
tracks_df = tracks_df[tracks_df["year"] >= 2000]
# features
# - chart_status  => [0: never charts, 1: has charted, 2: charted top 10]
# - award_status  => [0: never nominated, 1: nominated, 2: won]
# - artist_status => [0: never nominated, 1: previously nominated, 2: previous winner]

def chart_status(r):
    # should chart status account for album or artist charting?
    return 2 if r["track_chart_peak"] <= 25\
        else 1 if r["track_chart_peak"] > 0\
        else 0


def award_status(r):
    # should award status account for album or artist winning?
    return 2 if r["recording_award_winner"] >= 1\
        else 1 if r["recording_award_nominee"] >= 1\
        else 0


tracks_df["chart_status"] = tracks_df.apply(chart_status, axis=1)
tracks_df["award_status"] =  tracks_df.apply(award_status, axis=1)
# df["artist_status"] = df.apply(artist_status, axis=1)

print(tracks_df.shape)
tracks_df.head()

(14967, 69)


,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,duration_ms,...,artist_releases,artist_chart_months,artist_chart_months_recently,artist_releases_recently,genre,label_name,major_label,label_matched,chart_status,award_status
0,10iesoGb4mCYTcur1QfdO9,JCVD,Rien 100 Rien,2GLuHlc49dJKY8yzxUZb8p,Jul,"[""3IW7ScrzXmPvZhB27hmfgy""]",2,1,True,194266,...,39.0,0.0,0.0,33.0,rap marseille,D’Or et de Platine,0.0,1.0,0,0
1,0DlHQxrVijRqVUfuS6PKaY,Deixa de Onda (Porra Nenhuma),Deixa de Onda (Porra Nenhuma),1fjkJADMiiZWMDusZSMNix,"DENNIS, Xamã, LUDMILLA","[""6xlRSRMLgZbsSNd0BMobwy"", ""5YwzDz4RJfTiMHS4td...",1,1,True,184999,...,158.0,0.0,0.0,118.0,funk das antigas,Sony Music,1.0,1.0,0,0
2,2QtM6ZVLx13TBOO1iwmXYy,Cubra Libre Bar - Café Buddha del Mar Mix,Hotel Paradise del Mar (Chill Out Lounge Café ...,3wsWiiFbTF3TpC41yStCTL,Mark Bareilles,"[""22PKRoZlEMpKexRXSLStVP""]",12,1,False,332068,...,0.0,0.0,0.0,0.0,other,NaN,NaN,NaN,0,0
3,27hFQQS3cVUmIK3ser5bpu,More Than Gravity,More Than Gravity,4kcSIHj40oMyfHgMG6XAmO,Colin & Caroline,"[""2Y4125mDrgQyPD7dt76Rtm""]",1,1,False,266078,...,0.0,0.0,0.0,0.0,acoustic cover,NaN,NaN,NaN,0,0
4,1YcqYmviKagJ68DOmjSWQW,Tojo Tojo,Tojo Tojo,58NV8EvCFPHWwi7HHdVqSy,"Nik Tendo, Karlo","[""28oxY7ZfDt1BJFf8FVfpEh"", ""6gvVFtkTV4OCyQSCzG...",1,1,True,197010,...,81.0,0.0,0.0,68.0,czsk hip hop,W.C. Records,0.0,1.0,0,0


In [24]:
print(tracks_df[tracks_df["artist_matched"] == 0].shape)
print(tracks_df[(tracks_df["artist_matched"] == 0) & (tracks_df["track_chart_months"] >= 1)].shape)
print(tracks_df[tracks_df["label_matched"] != 1].shape)
tracks_df[(tracks_df["artist_matched"] == 0) & (tracks_df["track_chart_months"] >= 1)]

(72, 69)
(71, 69)
(8604, 69)


,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,duration_ms,...,artist_releases,artist_chart_months,artist_chart_months_recently,artist_releases_recently,genre,label_name,major_label,label_matched,chart_status,award_status
534,2UM93jKUu4r3wYujXSczbM,Last Night - Made famous by P Diddy Ft. Keyshi...,2007 Hot Hits vol 3,1qTZxNtKBc5NuO3FDtlBl2,,"[""0yHtl7OPYS3CaDLsOzFFw0""]",1,1,False,268761,...,0.0,0.0,0.0,0.0,other,NaN,NaN,NaN,2,0
598,2htzIJ4B3XBJteChnc1H3m,Das geht ab - Wir feiern die ganze Nacht - Atz...,Der Original Bierkönig DJ Chriss Tuxi Präsenti...,1e2JmLDonfxWwjia59Nfj1,,"[""38KSpOSYnwQX9zcOcVKdym"", ""4EAaUVCNTVA5rvmdyx...",3,1,False,209230,...,0.0,0.0,0.0,0.0,other,NaN,NaN,NaN,1,0
625,55yWXzQRjFqKea0mgxY0K7,2012 (If the World Would End) - Radio Mix,Smile,1pHPfwu4vFFyEwJK1rSuvA,,"[""24Sxfn1uAoJmuR9N72drt9"", ""5zntHEBXOIjmsrPNkd...",8,1,False,166963,...,0.0,0.0,0.0,0.0,other,NaN,NaN,NaN,1,0
680,6IFOw9Ck9mWHRQgw6QrPYx,Music (feat. Marvin Gaye),Music,7mUzxkBleB7QiVUw6wJsXU,,"[""2VX0o9LDIVmKIgpnwdJpOJ"", ""3koiLjNrgRTNbOwViD...",4,1,True,223133,...,0.0,0.0,0.0,0.0,other,NaN,NaN,NaN,1,0
992,17rwMsRgXow9SXnUv6Crrq,Haus am See (Tribute To Peter Fox),Deutsche Klub Musik Hits,7epov5fKkbtHMZtySqFugp,,"[""1oTxfI1gMBnoDMtPzrqgWO""]",6,1,False,225843,...,0.0,0.0,0.0,0.0,other,NaN,NaN,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15733,0v9wroQ8upZXY63IrpvL5u,Blah Blah Blah (Made Famous by Ke$ha feat. 3OH!3),100 Dance Beats For DJs & Singers,27Ncg4roNEGgvPehJfWNw4,,"[""6WJseLnjdHlIirNj1d3TLa""]",11,2,False,172564,...,0.0,0.0,0.0,0.0,other,NaN,NaN,NaN,2,0
15793,0C8PBBobLj0eeUQMWbwpSj,Around the World,Hits 2001 - Abra o Volume,5To2tM5XzqpZgNHNvCFsti,,"[""6BfoBSjpisCQJ2jzeMfnXa""]",7,1,False,312354,...,0.0,0.0,0.0,0.0,other,NaN,NaN,NaN,1,0
15925,4J6zcs7YMYiKBLjJUPMHLc,About You Now,Noughties Music,6Zx6kXuqFy9igTJ3oNesvq,,"[""7rZNSLWMjTbwdLNskFbzFf""]",6,1,False,212226,...,0.0,0.0,0.0,0.0,other,NaN,NaN,NaN,2,0
16015,3hiRNpEwJBKzYHM0LaP1E7,Stan (Made Famous By Eminem & Dido),Tunes Now: Hits,7AYPLkS0ZfoqDYEXVG7CFi,,"[""5itJHkE0MNCpDbj3SSwIL8""]",12,2,False,333741,...,0.0,0.0,0.0,0.0,other,NaN,NaN,NaN,2,0


In [20]:
print(f"no. tracks: {len(tracks_df)}")
print(f"no. chart2000 songs: {len(song_chart_df)}")
print(f"no. spotify chart songs: {len(spotify_chart_df)}")
print(f"no. grammy songs: {len(recording_nominations_df)}")
print(f"no. tracks w/out features: {(tracks_df['audio_features'] == 0).sum()}")
# print(f"no. track features w/out nominations: {tracks_df['spotify_track_id'].isnull().sum()}")
print(f"no. tracks w/out chart position: {len(tracks_df[(tracks_df['track_chart_months'].isnull()) & (tracks_df['recording_award_nominee'] > 0)])}")
print(f"no. charted songs w/ nominations: {len(tracks_df[(~tracks_df['track_chart_months'].isnull()) & (tracks_df['recording_award_nominee'] > 0)])}")
print(f"no. songs w/out nomination or chart: {len(tracks_df[(~tracks_df['track_chart_months'].isnull()) & (tracks_df['recording_award_nominee'] == 0)])}")

tracks_df.to_csv(data_path / "tracks.csv.gz", compression="gzip", index=False)
tracks_df.to_parquet(data_path / "tracks.pq", index=False)

tracks_df.groupby(["chart_status", "award_status"]).count()

no. tracks: 14967
no. chart2000 songs: 12900
no. spotify chart songs: 15000
no. grammy songs: 580
no. tracks w/out features: 2
no. tracks w/out chart position: 0
no. charted songs w/ nominations: 457
no. songs w/out nomination or chart: 14510


id   name  album  album_id  artists  artist_ids  \
chart_status award_status                                                       
0            0             11460  11460  11460     11460    11460       11460   
             1               211    211    211       211      211         211   
             2                37     37     37        37       37          37   
1            0              1525   1525   1525      1525     1525        1525   
             1                41     41     41        41       41          41   
             2                 3      3      3         3        3           3   
2            0              1525   1525   1525      1525     1525        1525   
             1               141    141    141       141      141         141   
             2                24     24     24        24       24          24   

                           track_number  disc_number  explicit  duration_ms  \
chart_status award_status                                                     
0            0                    11460        11460     11460        11460   
             1                      211          211       211          211   
             2                       37           37        37           37   
1            0                     1525         1525      1525         1525   
             1                       41           41        41           41   
             2                        3            3         3            3   
2            0                     1525         1525      1525         1525   
             1                      141          141       141          141   
             2                       24           24        24           24   

                           ...  artist_nominations  artist_wins  \
chart_status award_status  ...                                    
0            0             ...               11460        11460   
             1             ...                 211          211   
             2             ...                  37           37   
1            0             ...                1525         1525   
             1             ...                  41           41   
             2             ...                   3            3   
2            0             ...                1525         1525   
             1             ...                 141          141   
             2             ...                  24           24   

                           artist_releases  artist_chart_months  \
chart_status award_status                                         
0            0                       11460                11460   
             1                         211                  211   
             2                          37                   37   
1            0                        1525                 1525   
             1                          41                   41   
             2                           3                    3   
2            0                        1525                 1525   
             1                         141                  141   
             2                          24                   24   

                           artist_chart_months_recently  \
chart_status award_status                                 
0            0                                    11460   
             1                                      211   
             2                                       37   
1            0                                     1525   
             1                                       41   
             2                                        3   
2            0                                     1525   
             1                                      141   
             2                                       24   

                           artist_releases_recently  genre  label_name  \
chart_status award_status                                           

In [18]:
tracks_df[["track_chart_months", "recording_award_nominee"]][(~tracks_df['track_chart_months'].isnull()) & (tracks_df['recording_award_nominee'] == 0)]

,track_chart_months,recording_award_nominee
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
16123,0.0,0.0
16124,1.0,0.0
16125,0.0,0.0
16127,0.0,0.0


In [19]:
track_features_full_df = df[[
    'id',
    'name',
    'album',
    'album_id',
    'artists',
    'artist_ids',
    'track_number',
    'disc_number',
    'explicit',
    'duration_ms',
    'year',
    'release_date',
    'popularity',
    'isrc',
    # spotify audio features
    'audio_features', # has audio features
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'time_signature',
    # artist
    'artist_genre',
    'artist_gender',
    'artist_country',
    'artist_hometown',
    'artist_begin',
    'artist_total_releases'
    # charts2000
    'months', # num months on charts
    'indicativerevenue',
    'position_min',
    'position_mean',
    'position_median',
    # grammy award result for this song
    'recording_award_category',
    'recording_award_nominee',
    'recording_award_winner',
    # grammy award result for this song
    'album_award_category',
    'album_award_nominee',
    'album_award_winner',
    # grammy nomination history for artist
    'artist_total_nominations',
    'artist_total_wins',
    'artist_first_nomination',
    'artist_first_win',
    'artist_previous_nominations',
    'artist_previous_wins',
    # quick categories
    'chart_status',
    'award_status',
    'artist_status'
]]

track_features_full_df.to_csv("../../data/02-track_features_balanced_wide.csv.gz", compression="gzip", index=False)
track_features_full_df.to_parquet("../../data/02-track_features_balanced_wide.pq", index=False)

track_features_slim_df = df[[
    'id',
    'name',
    'album',
    'album_id',
    'artists',
    'artist_ids',
    'track_number',
    'disc_number',
    'explicit',
    'duration_ms',
    'year',
    'release_date',
    'popularity',
    # spotify audio features
    'audio_features', # has audio features
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'time_signature',
    # artist
    'artist_genre',
    'artist_gender',
    'artist_country',
    'artist_hometown',
    'artist_begin',
    'artist_total_releases',
    # quick categories
    'chart_status',
    'award_status',
    'artist_status'
]]

track_features_slim_df.to_csv("../../data/02-track_features_balanced.csv.gz", compression="gzip", index=False)
track_features_slim_df.to_parquet("../../data/02-track_features_balanced.pq", index=False)

NameError: name 'df' is not defined

In [ ]:
track_df = pd.read_parquet(data_path / "tracks.pq")
print(track_df[track_df["artist_matched"] == 0].shape)
print(track_df[track_df["label_matched"] == 0].shape)

In [ ]:
tracks_df = pd.read_parquet(data_path / "tracks.pq")
print(tracks_df[tracks_df["artist_matched"] == 0].shape)
print(tracks_df[tracks_df["label_matched"] == 0].shape)